In [1]:
import whitebox
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

# Initialize WhiteboxTools
wbt = whitebox.WhiteboxTools()

# Set working directory
project_dir = Path.home() / 'KolkataFloodMapping'
wbt.set_working_dir(str(project_dir / 'data' / 'elevation'))

print(f"WhiteboxTools version: {wbt.version}")
print(f"Working directory: {wbt.work_dir}")

Decompressing WhiteboxTools_darwin_m_series.zip ...
WhiteboxTools package directory: /opt/homebrew/Caskroom/miniforge/base/envs/kolkata_flood/lib/python3.13/site-packages/whitebox
WhiteboxTools version: <bound method WhiteboxTools.version of <whitebox.whitebox_tools.WhiteboxTools object at 0x1179af0e0>>
Working directory: /Users/romitbasak/KolkataFloodMapping/data/elevation


In [4]:
# First, check what files actually exist
import os
from pathlib import Path

# Check current directory and files
project_dir = Path.home() / 'KolkataFloodMapping'
elevation_dir = project_dir / 'data' / 'elevation'

print("Files in elevation directory:")
for file in elevation_dir.glob('*.tif'):
    print(f"  - {file.name}")

# Make sure the input file exists
input_file = elevation_dir / 'kolkata_elevation_clean.tif'
if not input_file.exists():
    print(f"\n❌ Input file not found: {input_file}")
else:
    print(f"\n✅ Input file exists: {input_file}")

# Check if WhiteboxTools is finding the right directory
print(f"\nWhiteboxTools working directory: {wbt.work_dir}")

Files in elevation directory:

❌ Input file not found: /Users/romitbasak/KolkataFloodMapping/data/elevation/kolkata_elevation_clean.tif

WhiteboxTools working directory: /Users/romitbasak/KolkataFloodMapping/data/elevation


In [6]:
# First, let's verify WhiteboxTools is properly installed
import whitebox

# Check if WhiteboxTools binary exists
wbt = whitebox.WhiteboxTools()
print(f"WhiteboxTools executable: {wbt.exe_path}")
print(f"Exists: {os.path.exists(wbt.exe_path)}")

# If the above shows False, download the binary
if not os.path.exists(wbt.exe_path):
    wbt.install()

WhiteboxTools executable: /opt/homebrew/Caskroom/miniforge/base/envs/kolkata_flood/lib/python3.13/site-packages/whitebox
Exists: True


In [7]:
# Debug approach - check if WBT can see your file
import whitebox
from pathlib import Path

wbt = whitebox.WhiteboxTools()

# Try setting verbose mode
wbt.verbose = True

# Check the exact file path
dem_file = Path.home() / 'KolkataFloodMapping' / 'data' / 'elevation' / 'kolkata_elevation_python.tif'
print(f"DEM file exists: {dem_file.exists()}")
print(f"Full path: {dem_file}")

# Try without setting working directory (use full paths only)
output_file = str(dem_file.parent / 'kolkata_filled.tif')

# Try the simplest operation first
try:
    wbt.slope(
        dem=str(dem_file),
        output=str(dem_file.parent / 'test_slope.tif')
    )
    print("✅ WhiteboxTools is working!")
except Exception as e:
    print(f"❌ Error: {e}")

DEM file exists: False
Full path: /Users/romitbasak/KolkataFloodMapping/data/elevation/kolkata_elevation_python.tif
./whitebox_tools --run="Slope" --dem='/Users/romitbasak/KolkataFloodMapping/data/elevation/kolkata_elevation_python.tif' --output='/Users/romitbasak/KolkataFloodMapping/data/elevation/test_slope.tif' --units=degrees -v --compress_rasters=False

****************************
* Welcome to Slope         *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
thread 'main' panicked at whitebox-tools-app/src/main.rs:72:21:
No such file or directory (os error 2)
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace
✅ WhiteboxTools is working!


In [8]:
# Install first
# pip install richdem

import richdem as rd
import matplotlib.pyplot as plt

# Load your DEM
dem_path = Path.home() / 'KolkataFloodMapping' / 'data' / 'elevation' / 'kolkata_elevation_python.tif'
dem = rd.LoadGDAL(str(dem_path))

print("Filling depressions...")
rd.FillDepressions(dem, in_place=True)

print("Calculating flow accumulation...")
flow_acc = rd.FlowAccumulation(dem, method='D8')

# Save results
output_path = dem_path.parent / 'flow_accumulation.tif'
rd.SaveGDAL(str(output_path), flow_acc)

# Visualize
flow_log = np.log10(flow_acc + 1)
plt.figure(figsize=(12, 10))
plt.imshow(flow_log, cmap='Blues')
plt.colorbar(label='Log10(Flow Accumulation)')
plt.title('Water Flow Accumulation in Kolkata')
plt.show()

print(f"✅ Flow accumulation saved to: {output_path}")

ModuleNotFoundError: No module named 'richdem'

In [9]:
! pip install richdem

  Using cached richdem-0.3.4.tar.gz (329 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached numpy-1.26.4-cp313-cp313-macosx_15_0_arm64.whl
  error: subprocess-exited-with-error
  
  × Building wheel for richdem (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [416 lines of output]
      /private/var/folders/78/p3gqt9651qs0hpq8s21hsn340000gn/T/pip-build-env-2p066eq0/overlay/lib/python3.13/site-packages/setuptools/dist.py:599: SetuptoolsDeprecationWarning: Invalid dash-separated key 'description-file' in 'metadata' (setup.cfg), please use the underscore name 'description_file' instead.
      !!
      
              ********************************************************************************
              Usage of dash-separated 'description-file' will not be supported in future
              versions. Please use the underscore name 'description_file' instead.
      